# Estimate a BLP model using Pyblp package
## This script is used for familirizing the BLP codes in Python
### Some variable names should be carefully treated, like prices, shares,demand_instruments#

In [3]:
import pyblp
import numpy as np
import pandas as pd

/Users/limingyi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/limingyi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [4]:
# some settings
pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'1.1.0'

## First of all, we estimate a  plain logit version of BLP
$u_{ijt} = \alpha p_{jt} + x_{jt}\beta + \xi_{jt} + \epsilon_{ijt}.$

In this case, $\delta_{jt} = \alpha p_{jt} +x_{jt}\beta + \xi_{jt}$ while $\mu_{ijt} = 0$,and $u_{ijt} = \delta_{jt} + \epsilon_{ijt}$.
Note that there is no nonlinear coefficients.

Following the forumula of BLP, $s_{jt} = \int_{A_{jt}}dP^*(\epsilon) = \frac{\exp(\alpha p_{jt} + x_{jt}\beta + \xi_{jt} )}{1 + \Sigma_l ( \alpha p_{lt} + x_{j=lt}\beta + \xi_{lt}) }$

After taking logs
$$
\log (s_{jt}) = \alpha p_{jt} + x_{jt} \beta + \xi_{jt} - \log (\Sigma_k \exp(\alpha p_{kt} + x_{kt} \beta + \xi_{kt}))$$ and 
$$
\log(s_{0t}) = -\log (\Sigma_k \exp(\alpha p_{kt} + x_{kt} \beta + \xi_{kt}))
$$

Hence, by differencing the above we get:
$$
\log(s_{jt}) - \log(s_{0t}) = \alpha p_{jt} + x_{jt} \beta + \xi_{jt}
$$
Because the left-hand side is data, we can estiamte this model using an linear IV GMM

In [5]:
# Data load
product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)

In [6]:
product_data.columns

Index(['market_ids', 'city_ids', 'quarter', 'product_ids', 'firm_ids',
       'brand_ids', 'shares', 'prices', 'sugar', 'mushy',
       'demand_instruments0', 'demand_instruments1', 'demand_instruments2',
       'demand_instruments3', 'demand_instruments4', 'demand_instruments5',
       'demand_instruments6', 'demand_instruments7', 'demand_instruments8',
       'demand_instruments9', 'demand_instruments10', 'demand_instruments11',
       'demand_instruments12', 'demand_instruments13', 'demand_instruments14',
       'demand_instruments15', 'demand_instruments16', 'demand_instruments17',
       'demand_instruments18', 'demand_instruments19'],
      dtype='object')

In [95]:
# Construct a `PROBLEM'
# Remember to name price as `prices', share as `shares' in the dataset. 
# Otherwise, the `PROBLEM' will not understand that price is endogenous
# Formulation refers to page 105 in the manual 


In [10]:
logit_formulation = pyblp.Formulation('prices',absorb = 'C(product_ids)')
problem = pyblp.Problem(logit_formulation, product_data)
logit_results = problem.solve()

In [11]:
logit_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix
Step    Value    Shares   Condition Number
----  ---------  -------  ----------------
 2    +1.9E+02      0         +5.7E+07    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
  prices  
----------
 -3.0E+01 
(+1.0E+00)

In [33]:
###### Random Coefficient ########

In [26]:
product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)

In [14]:
product_data.head()

,market_ids,city_ids,quarter,product_ids,firm_ids,brand_ids,shares,prices,sugar,mushy,...,demand_instruments10,demand_instruments11,demand_instruments12,demand_instruments13,demand_instruments14,demand_instruments15,demand_instruments16,demand_instruments17,demand_instruments18,demand_instruments19
0,C01Q1,1,1,F1B04,1,4,0.012417,0.072088,2,1,...,2.116358,-0.154708,-0.005796,0.014538,0.126244,0.067345,0.068423,0.034800,0.126346,0.035484
1,C01Q1,1,1,F1B06,1,6,0.007809,0.114178,18,1,...,-7.374091,-0.576412,0.012991,0.076143,0.029736,0.087867,0.110501,0.087784,0.049872,0.072579
2,C01Q1,1,1,F1B07,1,7,0.012995,0.132391,4,1,...,2.187872,-0.207346,0.003509,0.091781,0.163773,0.111881,0.108226,0.086439,0.122347,0.101842
3,C01Q1,1,1,F1B09,1,9,0.005770,0.130344,3,0,...,2.704576,0.040748,-0.003724,0.094732,0.135274,0.088090,0.101767,0.101777,0.110741,0.104332
4,C01Q1,1,1,F1B11,1,11,0.017934,0.154823,12,0,...,1.261242,0.034836,-0.000568,0.102451,0.130640,0.084818,0.101075,0.125169,0.133464,0.121111


In [27]:
X1_formulation = pyblp.Formulation('prices', absorb = 'C(product_ids)')
X2_formulation = pyblp.Formulation('1 + prices + sugar + mushy')
product_formulation = (X1_formulation,X2_formulation)

In [219]:
# Monte-Carlo draws from a pseudo-random multivariate normal distribution
# In this exercise, we do not have individual i's characteristics.
# Assume individual characteristics follow a multi-normal distribution
mc_int = pyblp.Integration('monte_carlo', size = 50, specification_options={'seed':0})

In [225]:
mc_problem = pyblp.Problem(product_formulation, product_data, integration=mc_int)

In [25]:
bfgs = pyblp.Optimization('bfgs',{'gtol':1e-4})
# Optimization methods see: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize

In [223]:
res1 = mc_problem.solve(sigma = np.ones((4,4)), optimization=bfgs)

The model may be under-identified. The total number of unfixed parameters is 4, which is more than the total number of moments, 1. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.


In [224]:
res1

Problem Results Summary:
GMM   Objective  Gradient      Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step    Value      Norm    Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  ---------  --------  --------------  --------------  -------  ----------------  -----------------
 2    +2.5E-31   +1.5E-16     -6.1E-09        +2.8E-08        0         +1.0E+00          +2.2E+18     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:00:04       Yes          0             4           766         2704    

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:      1         prices    |  Sigma Squared:      1         prices  
------  ----------  ----------  |  --------------  ----------  ----------
  1      +1.0E+00    

## The random-coefficient Utility looks like:
$$
u_{njt} = \delta_{nj} + \mu_{njt} + \epsilon_{njt}
$$
where 
$$
\delta_{jt} = -\beta price_{jt} + Brand~FE_j\\
\mu_{njt} = [1, price_{jt}, sugar_{jt}, mushy_{jt}] [\Pi D_i + \Sigma v_i]\\
D_i = [Income, Income^2, Age,Child]'
$$

In [28]:
agent_data = pd.read_csv(pyblp.data.NEVO_AGENTS_LOCATION)

In [29]:
agent_formulation = pyblp.Formulation('0 + income + income_squared + age + child')

In [36]:
nevo_problem = pyblp.Problem(product_formulation, product_data, agent_formulation, agent_data)
# mc_int = pyblp.Integration('monte_carlo', size = 20, specification_options={'seed':0})
# nevo_problem = pyblp.Problem(product_formulation, product_data, agent_formulation, agent_data,integration=mc_int)

In [37]:
nevo_problem

Dimensions:
 T    N     F    I     K1    K2    D    MD    ED 
---  ----  ---  ----  ----  ----  ---  ----  ----
94   2256   5   1880   1     4     4    20    1  

Formulations:
       Column Indices:           0           1           2      3  
-----------------------------  ------  --------------  -----  -----
 X1: Linear Characteristics    prices                              
X2: Nonlinear Characteristics    1         prices      sugar  mushy
       d: Demographics         income  income_squared   age   child

In [40]:
# Initialize nonlinear parameters 

# Pi
initial_sigma = np.diag([0.3, 2.4, 0.01, 0.2])

# Sigma
# zero means not updating these values and keeping them 0
initial_pi = np.array([
[ 5.4819,  0,      0.2037,  0     ],
[15.8935, -1.2000, 0,       2.6342],
[-0.2506,  0,      0.0511,  0     ],
[ 1.2650,  0,     -0.8091,  0     ]
])

In [258]:
# Also, we can use other initialization methods
initial_pi = np.random.normal(size = initial_pi.shape)

In [39]:
# We have 20 IVs in this dataset, which induces 20 moment conditions, 
# are incoporated into a target function using a sandwich form

In [42]:
tighter_bfgs = pyblp.Optimization('bfgs',{'gtol':1e-5})

In [46]:
# Initialize and solve
model_results = nevo_problem.solve(initial_sigma,initial_pi,optimization=tighter_bfgs)

In [303]:
results_list = []
#    results_list.append(['gradient', model_results.gradient[0][0], 0])
results_list.append(['objective', model_results.objective[0][0], 0])
results_list.append(['minutes_to_compute', model_results.cumulative_total_time/60, 0])
results_list.append(['converged', float(model_results.converged), 0])
df_optim = pd.DataFrame(results_list, columns = ['name', 'estimate', 'estimate_se'])

In [305]:
results_list

[['objective', 0.03104599903587035, 0],
 ['minutes_to_compute', 19.226098513603212, 0],
 ['converged', 1.0, 0]]